In [1]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [2]:
input_ = pd.read_excel('./file/2.make.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Year'] = input_['Year'].astype(int)

print(f'总数量：{len(input_)}')
input_['Make'].tolist()

总数量：4924


[1971,
 1970,
 1969,
 2001,
 2000,
 1999,
 1998,
 1997,
 1996,
 1995,
 1994,
 1993,
 1992,
 1974,
 1973,
 1972,
 1971,
 1970,
 1969,
 1993,
 1992,
 2008,
 2007,
 2006,
 2005,
 2004,
 2003,
 2025,
 2024,
 2023,
 2022,
 2021,
 2020,
 2019,
 2018,
 2017,
 2016,
 2015,
 2014,
 2013,
 2012,
 2011,
 2010,
 2009,
 2008,
 2007,
 2006,
 2005,
 2004,
 2003,
 2002,
 2001,
 2000,
 1999,
 1998,
 1997,
 1996,
 1995,
 1994,
 1993,
 1992,
 1991,
 1990,
 1989,
 1988,
 1987,
 1986,
 2006,
 2005,
 2004,
 2003,
 2002,
 2001,
 2000,
 1999,
 1998,
 1997,
 1996,
 1995,
 1994,
 1993,
 1990,
 1989,
 1988,
 2017,
 2016,
 2015,
 2014,
 2013,
 2012,
 2023,
 2022,
 2021,
 2020,
 2019,
 2018,
 2017,
 2016,
 2015,
 1995,
 1994,
 1993,
 1992,
 1991,
 1990,
 1989,
 1988,
 1987,
 1986,
 1985,
 1984,
 1983,
 1982,
 1981,
 1980,
 1979,
 1978,
 1977,
 1976,
 1975,
 1974,
 1973,
 1972,
 1971,
 1970,
 2024,
 2023,
 2022,
 2021,
 2017,
 2016,
 2015,
 2014,
 2013,
 2012,
 2011,
 2010,
 2009,
 2008,
 2007,
 2013,
 2012,
 2011,

In [3]:
a = 0

a

0

In [4]:
crawler_status = 'error'

crawler_status

'error'

In [5]:
import requests
import json

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

data = {'cssClass': 'form-control',
        'dropdownType': '2',
        'selectedValue': '',
        'vehicleAttributes': {'Make': f'''{input_.loc[a, 'Make']}''',
                              'MakeID': f'''{input_.loc[a, 'Make Code']}''',
                              'ModelID': 0,
                              'SkinID': '1',
                              'Year': f'''{input_.loc[a, 'Year']}'''}}

b = 0
while True:
    b += 1
    print(b)
    try:
        resp = requests.post('https://www.dormanproducts.com/ApplicationGuideService.asmx/BuildDropdown',
                             data=json.dumps(data),
                             headers=get_header(**{'Content-Type': 'application/json; charset=UTF-8'}),
                             proxies=get_proxy(),
                             timeout=(10, 10))
        
        if resp.status_code == 200:
            break
    except KeyboardInterrupt:
        break
    except:
        continue

resp

1


<Response [200]>

In [6]:
json_ = resp.json()

json_

{'d': '<select id="modelDropdown" name="modelDropdown" onchange="return onDropdownChanged(3, modelDropdown.selectedIndex == 0, \'parttypeDropdownElement\');" class="form-control">\n<option value="0" selected="1" >{0}</option>\n<option value="21378"  >Shelby Cobra</option>\n</select>'}

In [7]:
from bs4 import BeautifulSoup
from lxml import etree

soup = BeautifulSoup(json_['d'], 'lxml')
html = etree.HTML(str(soup))

with open('./model.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<html>
 <body>
  <select class="form-control" id="modelDropdown" name="modelDropdown" onchange="return onDropdownChanged(3, modelDropdown.selectedIndex == 0, 'parttypeDropdownElement');">
   <option selected="1" value="0">
    {0}
   </option>
   <option value="21378">
    Shelby Cobra
   </option>
  </select>
 </body>
</html>



In [8]:
list_model_code = html.xpath('//select[@id="modelDropdown"]/option/@value')[1:]

list_model_code

['21378']

In [9]:
list_model = html.xpath('//select[@id="modelDropdown"]/option/text()')[1:]

list_model

['Shelby Cobra']

In [10]:
len(list_model) == 0

False

In [11]:
df_temp = pd.DataFrame({'Make': input_.loc[a, 'Make'],
                        'Make Code': input_.loc[a, 'Make Code'],
                        'Model': list_model,
                        'Model Code': list_model_code,
                        'Year': input_.loc[a, 'Year']})

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,Make,Make Code,Model,Model Code,Year
0,AC,457,Shelby Cobra,21378,1971


In [12]:
crawler_status = 'ok'

crawler_status

'ok'

In [13]:
if crawler_status == 'error':
    output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

output_error

""


In [14]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Make', 'Model', 'Year'],
                                                ascending=[True, True, False],
                                                ignore_index=True)
    output_correct.to_excel('./test_model.xlsx', index=False)

output_correct

,Make,Make Code,Model,Model Code,Year
0,AC,457,Shelby Cobra,21378,1971


In [15]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['Make', 'Year'],
                                            ascending=[True, False],
                                            ignore_index=True)
    output_error.to_excel('./test_model_error.xlsx', index=False)

output_error

""
